In [ ]:
import re
import inspect
import json
from enum import Enum

from faker import Faker
from faker.providers import DynamicProvider

import demo_dataset.types.common as common
import demo_dataset.types.smart_hubs as smart_hubs
import demo_dataset.types.smart_switches as smart_switches
import demo_dataset.types.smart_bulbs as smart_bulbs
import demo_dataset.types.smart_thermostat as smart_thermostat
import demo_dataset.types.vacuum_robots as vacuum_robots
import demo_dataset.types.solar_panels as solar_panels

modules = [
    common,
    smart_bulbs,
    smart_hubs,
    smart_switches,
    smart_thermostat,
    vacuum_robots,
    solar_panels,
]

fake = Faker()

In [94]:
# Helper Function to transform camel case to sake case
def camel_to_snake(camel_str: str):
    snake_str = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', camel_str)
    return snake_str.lower()

In [95]:
providers = []
for module in modules:
    providers.extend([
        DynamicProvider(
            provider_name=f"smarthome_{module.__name__.split('.')[-1]}_{camel_to_snake(name)}",
            elements=list(map(lambda e: e.value, obj))
        ) for name, obj in inspect.getmembers(module) 
            if (
                inspect.isclass(obj) and
                issubclass(obj, Enum) and
                obj is not Enum 
                and obj.__module__ == module.__name__
            )
    ])


print(providers)
for provider in providers:
    fake.add_provider(provider)

[<faker.providers.DynamicProvider object at 0x7f9c6b411240>, <faker.providers.DynamicProvider object at 0x7f9c6b49eb00>, <faker.providers.DynamicProvider object at 0x7f9c6b49ea10>, <faker.providers.DynamicProvider object at 0x7f9c6b49ead0>, <faker.providers.DynamicProvider object at 0x7f9c6b49e7d0>, <faker.providers.DynamicProvider object at 0x7f9c6b49c820>, <faker.providers.DynamicProvider object at 0x7f9c6b49c970>, <faker.providers.DynamicProvider object at 0x7f9c6b49ca30>, <faker.providers.DynamicProvider object at 0x7f9c6b49c400>, <faker.providers.DynamicProvider object at 0x7f9c6b49c460>, <faker.providers.DynamicProvider object at 0x7f9c6b49c040>, <faker.providers.DynamicProvider object at 0x7f9c6b49d840>, <faker.providers.DynamicProvider object at 0x7f9c6b49cdf0>, <faker.providers.DynamicProvider object at 0x7f9c6b49c1f0>, <faker.providers.DynamicProvider object at 0x7f9c6b49c220>, <faker.providers.DynamicProvider object at 0x7f9c6b49c1c0>, <faker.providers.DynamicProvider object

In [96]:
fake.smarthome_solar_panels_panel_height()

1200

In [ ]:
from dataclasses import dataclass
import random
from typing import List, Optional

from demo_dataset.types.common import Color, CommunicationProtocol, EnergyEfficiencyClass, Platform, Shape
from demo_dataset.types.smart_bulbs import BulbFeatures, LightColor, RequiresHub, Socket
from demo_dataset.types.smart_hubs import HubFeatures
from demo_dataset.types.smart_switches import SwitchFeatures, SwitchStyle
from demo_dataset.types.smart_thermostat import CWire, RemoteTemperatureSensor, ReplaceableBatteries, ThermostatFeatures
from demo_dataset.types.solar_panels import SolarBrand, PanelLength, PanelHeight, PanelWidth
from demo_dataset.types.vacuum_robots import Dock, HardwareFeature, SoftwareFeature


@dataclass(kw_only=True, init=False)
class SmartProduct:
    # 10 long numeric string beginning with 11xxxxxxxx
    product_no: Optional[str] = None
    product_type: str
    name: Optional[str] = None
    brand: str
    price: float
    description: Optional[str] = None


@dataclass(kw_only=True)
class SmartSwitch(SmartProduct):
    platforms: List[Platform]
    communication_protocol: List[CommunicationProtocol]
    style: SwitchStyle
    features: SwitchFeatures

    def __init__(self):
        self.brand = fake.smarthome_common_brand()
        self.product_type = self.__class__.__name__
        self.price = random.choice([22.99, 29.99, 15.00, 50.00])
        self.platforms = list({fake.smarthome_common_platform() for _ in range(fake.random_int(3, 9))})
        self.communication_protocol = list({fake.smarthome_common_communication_protocol() for _ in range(fake.random_int(3, 6))})
        self.style = fake.smarthome_smart_switches_switch_style()
        self.features = list({fake.smarthome_smart_switches_switch_features() for _ in range(fake.random_int(3, 8))})


@dataclass(kw_only=True)
class SmartBulb(SmartProduct):
    platforms: List[Platform]
    communication_protocol: List[CommunicationProtocol]
    light_color: LightColor
    socket: Socket
    energy_efficiency_class: EnergyEfficiencyClass
    features: BulbFeatures
    requireshub: RequiresHub
    lifetime: int
    wattage: int
    brightness: int
    color_temp: int | None

    def __init__(self):
        self.brand = fake.smarthome_common_brand()
        self.product_type = self.__class__.__name__
        self.price = random.choice([22.99, 29.99, 8.00, 13.99, 18.99, 9.99, 28.99])
        self.platforms = list({fake.smarthome_common_platform() for _ in range(fake.random_int(3, 9))})
        self.communication_protocol = list({fake.smarthome_common_communication_protocol() for _ in range(fake.random_int(3, 6))})
        self.light_color = fake.smarthome_smart_bulbs_light_color()
        self.socket = fake.smarthome_smart_bulbs_socket()
        self.energy_efficiency_class = fake.smarthome_common_energy_efficiency_class()
        self.features = fake.smarthome_smart_bulbs_bulb_features()
        self.requireshub = fake.smarthome_smart_bulbs_requires_hub()
        self.lifetime = random.choice([10000, 15000, 20000, 25000, 30000])
        self.wattage = random.choice([6, 8, 10, 12,  60])
        self.brightness = random.choice([500, 600, 800, 900])
        self.color_temp = random.choice([2700, 3000, 4000, 6000]) if self.light_color != "RGB" else None


@dataclass(kw_only=True)
class SmartThermostat(SmartProduct):
    platforms: List[Platform]
    communication_protocol: List[CommunicationProtocol]
    color: Color
    shape: Shape
    features: List[ThermostatFeatures]
    cwire: CWire
    remote_temperature_sensor: RemoteTemperatureSensor
    replaceable_batteries: ReplaceableBatteries

    def __init__(self):
        self.brand = fake.smarthome_common_brand()
        self.product_type = self.__class__.__name__
        self.platforms = list({fake.smarthome_common_platform() for _ in range(fake.random_int(3, 9))})
        self.price = random.choice([100.00, 133.99, 250.00, 95.00, 99.99, 80.00, 75.00, 175.99])
        self.communication_protocol = list({fake.smarthome_common_communication_protocol() for _ in range(fake.random_int(3, 6))})
        self.color = fake.smarthome_common_color()
        self.shape = fake.smarthome_common_shape()
        self.features = list({fake.smarthome_smart_thermostat_thermostat_features() for _ in range(fake.random_int(3, 9))})
        self.cwire = fake.smarthome_smart_thermostat_cwire()
        self.remote_temperature_sensor = fake.smarthome_smart_thermostat_remote_temperature_sensor()
        self.replaceable_batteries = fake.smarthome_smart_thermostat_replaceable_batteries()


@dataclass(kw_only=True)
class SmartHub(SmartProduct):
    platforms: List[Platform]
    communication_protocol: List[CommunicationProtocol]
    features: List[HubFeatures]

    def __init__(self):
        self.brand = fake.smarthome_common_brand()
        self.product_type = self.__class__.__name__
        self.price = random.choice([55.00, 100.00, 155.00, 230.00, 190.00])
        self.platforms = list({fake.smarthome_common_platform() for _ in range(fake.random_int(3, 9))})
        self.communication_protocol = list({fake.smarthome_common_communication_protocol() for _ in range(fake.random_int(3, 6))})
        self.features = list({fake.smarthome_smart_hubs_hub_features() for _ in range(fake.random_int(1, 2))})


@dataclass(kw_only=True)
class VacuumRobot(SmartProduct):
    suction_strength: int
    hardware_features: List[HardwareFeature]
    software_features: List[SoftwareFeature]
    dock: Dock

    def __init__(self):
        self.brand = fake.smarthome_common_brand()
        self.product_type = self.__class__.__name__
        self.price = random.choice([200.00, 300.00, 400.00, 500.00, 600.00, 1000.00])
        self.suction_strength = random.choice([5300, 6000, 2700, 2000, 3000])
        self.hardware_features = list({fake.smarthome_vacuum_robots_hardware_feature() for _ in range(fake.random_int(2, 5))})
        self.software_features = list({fake.smarthome_vacuum_robots_software_feature() for _ in range(fake.random_int(1, 3))})
        self.dock = fake.smarthome_vacuum_robots_dock()



@dataclass(kw_only=True, init=False)
class SolarPanel(SmartProduct):
    brand: SolarBrand
    length: PanelLength
    height: PanelHeight
    width: PanelWidth
    max_power_output: int
    weight: int

    def __init__(self):
        self.brand = fake.smarthome_solar_panels_solar_brand()
        self.product_type = self.__class__.__name__
        self.length = fake.smarthome_solar_panels_panel_length()
        self.height = fake.smarthome_solar_panels_panel_height()
        self.width = fake.smarthome_solar_panels_panel_width()
        self.surface_area = self.length * self.height / 1000000
        self.max_power_output = random.choice([100, 200, 300, 400, 450])
        self.price = round((self.max_power_output * 0.01 * self.surface_area * random.choice([60, 75, 85]) * 0.2), 2)
        self.weight = round((self.surface_area * fake.random_int(16, 24)), 2)

In [98]:
import dataclasses

products = []
products.extend([dataclasses.asdict(SmartSwitch()) for _ in range(10)])
products.extend([dataclasses.asdict(SmartBulb()) for _ in range(30)])
products.extend([dataclasses.asdict(SmartThermostat()) for _ in range(10)])
products.extend([dataclasses.asdict(SmartHub()) for _ in range(10)])
products.extend([dataclasses.asdict(VacuumRobot()) for _ in range(10)])
products.extend([dataclasses.asdict(SolarPanel()) for _ in range(10)])

products

[{'product_no': None,
  'product_type': 'SmartSwitch',
  'name': None,
  'brand': 'Hometics',
  'price': 50.0,
  'description': None,
  'platforms': ['APPLE_HOME_KIT',
   'IFTTT',
   'SMART_THINGS',
   'ALEXA',
   'GOOGLE_HOME'],
  'communication_protocol': ['Z_WAVE', 'MATTER', 'WIFI', 'BLUETOOTH'],
  'style': 'SWITCH',
  'features': ['THREE_WAY',
   'MOTION_ACTIVATED',
   'TWO_POLE',
   'NEUTRAL_WIRE_NEEDED',
   'LOCAL_ONLY_AVAILABLE']},
 {'product_no': None,
  'product_type': 'SmartSwitch',
  'name': None,
  'brand': 'Intuit Home',
  'price': 29.99,
  'description': None,
  'platforms': ['ALEXA', 'APPLE_HOME_KIT', 'HUBITAT', 'SMART_THINGS'],
  'communication_protocol': ['ZIGBEE', 'Z_WAVE', 'MATTER'],
  'style': 'DIMMER',
  'features': ['NO_NEUTRAL_WIRE_NEEDED',
   'MOTION_ACTIVATED',
   'CEILING_FAN_SWITCH',
   'THREE_POLE',
   'TWO_POLE',
   'NEUTRAL_WIRE_NEEDED',
   'LOCAL_ONLY_AVAILABLE']},
 {'product_no': None,
  'product_type': 'SmartSwitch',
  'name': None,
  'brand': 'Intuit H

In [106]:
from langchain_ollama import OllamaLLM


llm = OllamaLLM(model="llama3.1:8b", temperature=0.8)

In [120]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["json_data"],
    template="""
    Du bist ein KI-Modell für die generierung von Produkttiteln. Deine Aufgabe ist es auf Grundlage übergebener Werte einen Titel für ein Produkt zu erzeugen, der Titel soll kurz und knapp auf basis des gegebenen JSON Objects erstellt werden.
    Gebe nur den Titel des Produkts aus ohne weiteren Kommentar. Sei kreativ beim erstellen des Produkttitels, wie ein Name für eine Produktfamilie und einer Generation . Verwende auf keinen Fall die Brand im Titel. Verwende auf keinen Fall den Preis im Titel. Verwende auf keinen Fall den product_type im Titel. Verwende auf keinen Fall den product_no im Titel. Verwende auf keinen Fall den description im Titel. Gebe niemals mehrere Titel oder alternative Titel aus. Generiere immer nur genau einen prägnanten Titel.
    

    Basierend auf den folgenden Werten, schreibe einen Produkttitel:
    {json_data}
    """
)

chain = prompt_template | llm

for idx, product in enumerate(products):
    chunks = []
    for chunk in chain.stream({"json_data": json.dumps(product)}):
        chunks.append(chunk)
    products[idx]["name"] = "".join(chunks)

products

[{'product_no': None,
  'product_type': 'SmartSwitch',
  'name': 'IntelliSwitch Evolution',
  'brand': 'Hometics',
  'price': 50.0,
  'description': None,
  'platforms': ['APPLE_HOME_KIT',
   'IFTTT',
   'SMART_THINGS',
   'ALEXA',
   'GOOGLE_HOME'],
  'communication_protocol': ['Z_WAVE', 'MATTER', 'WIFI', 'BLUETOOTH'],
  'style': 'SWITCH',
  'features': ['THREE_WAY',
   'MOTION_ACTIVATED',
   'TWO_POLE',
   'NEUTRAL_WIRE_NEEDED',
   'LOCAL_ONLY_AVAILABLE']},
 {'product_no': None,
  'product_type': 'SmartSwitch',
  'name': 'Intuit Home DIMMER 3.0 Smart',
  'brand': 'Intuit Home',
  'price': 29.99,
  'description': None,
  'platforms': ['ALEXA', 'APPLE_HOME_KIT', 'HUBITAT', 'SMART_THINGS'],
  'communication_protocol': ['ZIGBEE', 'Z_WAVE', 'MATTER'],
  'style': 'DIMMER',
  'features': ['NO_NEUTRAL_WIRE_NEEDED',
   'MOTION_ACTIVATED',
   'CEILING_FAN_SWITCH',
   'THREE_POLE',
   'TWO_POLE',
   'NEUTRAL_WIRE_NEEDED',
   'LOCAL_ONLY_AVAILABLE']},
 {'product_no': None,
  'product_type': 'Sma

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["json_data"],
    template="""
    Du bist ein KI-Modell für Produkttextgenerierung. Deine Aufgabe ist es auf Grundlage übergebener Werte Texte zu erzeugen, die ein Produkt auf basis des gegebenen JSON Objects beschreiben.
    Gebe nur den Text aus ohne weiteren Kommentar. Erstelle einen Fließtext ohne sich wiederholende Phrasen. 

    Basierend auf den folgenden Werten, schreibe einen Text:
    {json_data}
    """
)

chain = prompt_template | llm

for idx, product in enumerate(products):
    chunks = []
    for chunk in chain.stream({"json_data": json.dumps(product)}):
        chunks.append(chunk)
    products[idx]["description"] = "".join(chunks)